In [1]:
import os
import re
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')

In [8]:
df = pd.read_csv("data/Batch_4978433_batch_results.csv")
df.head(3)

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,Answer.oti.2,Answer.oti.3,Answer.otl.1,Answer.otl.2,Answer.otl.3,Answer.ovi.1,Answer.ovi.2,Answer.ovi.3,Approve,Reject
0,32W3UF2EZOC22EWSKMVNIRE349OC48,3UMJP5XW5JYSCFJZKNHNNHQQ1FFUF1,Compare the quality of two sequences of steps ...,Compare the quality of two sequences of steps ...,"Informativeness, Text and Image, Planning, Tas...",$0.10,Sun Jan 01 20:11:29 PST 2023,3,BatchId:4978433;OriginalHitTemplateId:928390905;,300,...,True,False,True,False,False,False,False,True,NaN,NaN
1,32W3UF2EZOC22EWSKMVNIRE349OC48,3UMJP5XW5JYSCFJZKNHNNHQQ1FFUF1,Compare the quality of two sequences of steps ...,Compare the quality of two sequences of steps ...,"Informativeness, Text and Image, Planning, Tas...",$0.10,Sun Jan 01 20:11:29 PST 2023,3,BatchId:4978433;OriginalHitTemplateId:928390905;,300,...,True,False,True,False,False,False,False,True,NaN,NaN
2,32W3UF2EZOC22EWSKMVNIRE349OC48,3UMJP5XW5JYSCFJZKNHNNHQQ1FFUF1,Compare the quality of two sequences of steps ...,Compare the quality of two sequences of steps ...,"Informativeness, Text and Image, Planning, Tas...",$0.10,Sun Jan 01 20:11:29 PST 2023,3,BatchId:4978433;OriginalHitTemplateId:928390905;,300,...,True,False,True,False,False,False,False,True,NaN,NaN


In [29]:
score_map = {}
for datatype in ["wikihow", "recipeqa"]:
    score_map[datatype] = {}
    df_datatype = df.loc[df['Input.datatype'] == datatype]
    for baseline in ["u-plan", "vgt-u-plan"]:
        df_baseline = df_datatype[df_datatype['Input.plan2_grid_path'].str.contains(baseline)]
        score_map[datatype][baseline] = {"oti":{}, "ovi":{}, "otl":{}, "opa":{}}
        for metric_type in ["oti", "ovi", "otl", "opa"]:
            ours = len(df_baseline.loc[df_baseline[f'Answer.{metric_type}.1'] == True]) # 1 - Sequence 1 is better
            equal = len(df_baseline.loc[df_baseline[f'Answer.{metric_type}.2'] == True]) # 2 - Tie
            base = len(df_baseline.loc[df_baseline[f'Answer.{metric_type}.3'] == True]) # 3 - Sequence 2 is better
            total = ours+equal+base
            score_map[datatype][baseline][metric_type] = {"win":0, "tie":0, "lose":0}
            score_map[datatype][baseline][metric_type]['win'] = round(ours/total*100, 2)
            score_map[datatype][baseline][metric_type]['tie'] = round(equal/total*100, 2)
            score_map[datatype][baseline][metric_type]['lose'] = round(base/total*100, 2)
score_map

{'wikihow': {'u-plan': {'oti': {'win': 40.85, 'tie': 42.25, 'lose': 16.9},
   'ovi': {'win': 30.67, 'tie': 41.33, 'lose': 28.0},
   'otl': {'win': 26.67, 'tie': 44.0, 'lose': 29.33},
   'opa': {'win': 37.84, 'tie': 27.03, 'lose': 35.14}},
  'vgt-u-plan': {'oti': {'win': 42.11, 'tie': 42.11, 'lose': 15.79},
   'ovi': {'win': 36.67, 'tie': 35.0, 'lose': 28.33},
   'otl': {'win': 30.0, 'tie': 43.33, 'lose': 26.67},
   'opa': {'win': 43.33, 'tie': 23.33, 'lose': 33.33}}},
 'recipeqa': {'u-plan': {'oti': {'win': 40.85, 'tie': 42.25, 'lose': 16.9},
   'ovi': {'win': 30.67, 'tie': 41.33, 'lose': 28.0},
   'otl': {'win': 26.67, 'tie': 44.0, 'lose': 29.33},
   'opa': {'win': 37.84, 'tie': 27.03, 'lose': 35.14}},
  'vgt-u-plan': {'oti': {'win': 42.11, 'tie': 42.11, 'lose': 15.79},
   'ovi': {'win': 36.67, 'tie': 35.0, 'lose': 28.33},
   'otl': {'win': 30.0, 'tie': 43.33, 'lose': 26.67},
   'opa': {'win': 43.33, 'tie': 23.33, 'lose': 33.33}}}}

In [34]:
for datatype in score_map.keys():
    print(datatype)
    for baseline in score_map[datatype].keys():
        print("{} & {} & {} & {} & {} & {} & {} & {} & {} & {} & {} & {} & {} \\\\".format(baseline, score_map[datatype][baseline]["oti"]["win"], score_map[datatype][baseline]["oti"]["tie"], score_map[datatype][baseline]["oti"]["lose"], score_map[datatype][baseline]["ovi"]["win"], score_map[datatype][baseline]["ovi"]["tie"], score_map[datatype][baseline]["ovi"]["lose"], score_map[datatype][baseline]["otl"]["win"], score_map[datatype][baseline]["otl"]["tie"], score_map[datatype][baseline]["otl"]["lose"], score_map[datatype][baseline]["opa"]["win"], score_map[datatype][baseline]["opa"]["tie"], score_map[datatype][baseline]["opa"]["lose"]))

wikihow
u-plan & 40.85 & 42.25 & 16.9 & 30.67 & 41.33 & 28.0 & 26.67 & 44.0 & 29.33 & 37.84 & 27.03 & 35.14 \\
vgt-u-plan & 42.11 & 42.11 & 15.79 & 36.67 & 35.0 & 28.33 & 30.0 & 43.33 & 26.67 & 43.33 & 23.33 & 33.33 \\
recipeqa
u-plan & 40.85 & 42.25 & 16.9 & 30.67 & 41.33 & 28.0 & 26.67 & 44.0 & 29.33 & 37.84 & 27.03 & 35.14 \\
vgt-u-plan & 42.11 & 42.11 & 15.79 & 36.67 & 35.0 & 28.33 & 30.0 & 43.33 & 26.67 & 43.33 & 23.33 & 33.33 \\
